# Python and GIS: Folium, Mapbox, and Shapely
* *Folium*: interactive map visualisation
* *Mapbox*: geocoding and isochrone api
* *Shapely*: calculating polygon's areas and intersections

<img src="maps_img.jpg" style="width:600px;">

Review these additional resources to learn more about the above.
- [Geopandas](https://geopandas.org/en/stable/)
- [Shapely](https://shapely.readthedocs.io/en/stable/manual.html)
- [Folium](https://python-visualization.github.io/folium/)
- [Mapbox API](https://docs.mapbox.com/api/overview/)
- [Isochrones](https://en.wikipedia.org/wiki/Isochrone_map)

### Overview

This notebook shows how to perform simple, yet fundamental, GIS operations: 

1. Mapbox's API is called using `request` and `json` for *geocoding* and generating *isochrones*.
2. GIS data is processed using `shapely` and `geopandas`. 
3. The data is then visualised using `folium` which generates interactive maps.

### Setup

We start by importing the necessary libraries and writing functions that will be used throughout the notebook.

In [228]:
# for data manipulation
import pandas as pd
import numpy as np

# for gis data manipulation
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# for MAPBOX API
import requests
import json

# for interactive maps
import folium

In [229]:
# < ---- REPLACE TOKEN BELOW ---- >
TOKEN = "<YOUR MAPBOX API TOKEN>"

In [230]:
# geocoding: address to coordinates
def get_coordinates(address):
    '''
    Geocoding function returning (lat,lon)
    Argument(s):
    -- address (str)
    Output:
    -- tuple/numeric
    '''

    # get request 
    req = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/"+
                           address.replace(" ","%20")+
                           ".json?access_token="+
                           TOKEN+
                           "&cachebuster=1615406248283&"+
                           "autocomplete=true")
    
    # get content
    content = req.content
    # load as json
    content = json.loads(content)
    
    # exract lon,lat
    lon,lat = content["features"][0]["center"]
    
    return lat, lon

# polygon from lat, lon, and distance
def get_isochrone(lat,lon,mins,v="driving"):
    '''
    Single Isochrone function returning polygon as list
    Argument(s):
    -- lat (numeric) latitude
    -- lon (numeric) longitude
    -- mins (numeric) minutes
    -- v (string) one of ["walking","cycling","driving"]
    Output:
    -- list (list/numeric)
    '''
    # get request
    req = requests.get("https://api.mapbox.com/isochrone/v1/mapbox/"+
                        v+"/"+str(lon)+"%2C"+str(lat)+
                       "?contours_minutes="+str(mins)+
                       "&contours_colors="+"54278f"+
                       "&polygons=true&generalize=0&"+
                       "access_token="+TOKEN)

    # get content
    content = req.content
    # load as json
    content = json.loads(content)

    # get coords
    shp = content['features'][0]['geometry']['coordinates']
    # flip lat lon
    shp = np.flip(np.array(shp[0]),axis=1)
    
    return shp

# polygon from lat, lon, and (multiple) distances
def get_isochrones(lat,lon,mins,v="driving"):
    '''
    Multiple Isochrones function returning polygons as list
    Argument(s):
    -- lat (numeric) latitude
    -- lon (numeric) longitude
    -- mins (numeric) minutes
    -- v (string) one of ["walking","cycling","driving"]
    Output:
    -- list (list/numeric)
    '''
    shp = []
    mins.sort(reverse=1)

    for m in mins:
        shp.append(get_isochrone(lat,lon,m,v))
        
    return shp

### Geocoding & Isochrones

Use the functions defined above to retrieve the data for coordinates (i.e. lat, lon) and the isochrones (polygons)

In [231]:
# addresses as strings
address= "via spadolini 10, milano"
address2= "via valtellina 40, milano"

# get coordinates from addresses
lat,lon = get_coordinates(address)
lat2,lon2 = get_coordinates(address2)

In [232]:
# minutes
mins = [5,15,10,20,30,40,50,60]
# colors
cols = ["black","pink","purple","red","orange","yellow","green","blue"]
# means of transportation
means = "walking"

# get isochrone polygons for both addresses: 2 sets of 8 isochrone polygons (16 in total)
shp = get_isochrones(lat, lon, mins,v=means)
shp2 = get_isochrones(lat2, lon2, mins,v=means)

### Interactive Maps

Visualise the isochrone polygons using `folium`, a python implementation of `leaflet.js`. 

In [233]:
# create map
mp = folium.Map([lat,lon],zoom_start=12)

# add isochrone polygon
for i in range(len(shp)):
    folium.Polygon(shp[i],
                   tooltip=mins[i],
                   label = mins[i],
                   fill_opacity=.3,
                   fill_color=cols[i],
                   stroke=0).add_to(mp)

# add isochrone polygon
for i in range(len(shp2)):
    folium.Polygon(shp2[i],
                   tooltip=mins[i],
                   label = mins[i],
                   fill_opacity=0.3,
                   fill_color=cols[i],
                   stroke=0).add_to(mp)
    

# add location marker
folium.Marker([lat,lon],tooltip=address).add_to(mp)

# show map
mp

### Polygon Overlap

Given multiple polygons, calculate the overlap, or intersection, and return the percentage of shared area.

In [234]:
# one polygon for each isochrone in shp
p0 = Polygon(shp[0])
p1 = Polygon(shp[1])
p2 = Polygon(shp[2])
p3 = Polygon(shp[3])
p4 = Polygon(shp[4])
p5 = Polygon(shp[5])
p6 = Polygon(shp[6])
p7 = Polygon(shp[7])
p8 = Polygon(shp2[0])
p9 = Polygon(shp2[1])
p10 = Polygon(shp2[2])
p11 = Polygon(shp2[3])
p12 = Polygon(shp2[4])
p13 = Polygon(shp2[5])
p14 = Polygon(shp2[6])
p15 = Polygon(shp2[7])

In [235]:
overlap = p0.intersection(p1)
print('Coverage: ', round(overlap.area/p0.area*100,2),'%')

Coverage:  71.52 %


### Geopandas

Geopandas can now be used to generate a `geopandas` data-frame to contain all polygons.

With a `geopandas` data-frame it is relatively easy to perform operations such as grouping together multiple polygons (i.e. dissolving). 

In [236]:
newdata = gpd.GeoDataFrame()
newdata['geometry'] = None

newdata.loc[0, 'geometry'] = p0
newdata.loc[1, 'geometry'] = p1
newdata.loc[2, 'geometry'] = p2
newdata.loc[3, 'geometry'] = p3
newdata.loc[4, 'geometry'] = p4
newdata.loc[5, 'geometry'] = p5
newdata.loc[6, 'geometry'] = p6
newdata.loc[7, 'geometry'] = p7
newdata.loc[8, 'geometry'] = p8
newdata.loc[9, 'geometry'] = p9
newdata.loc[10, 'geometry'] = p10
newdata.loc[11, 'geometry'] = p11
newdata.loc[12, 'geometry'] = p12
newdata.loc[13, 'geometry'] = p13
newdata.loc[14, 'geometry'] = p14
newdata.loc[15, 'geometry'] = p15

newdata

,geometry
0,"POLYGON ((45.48083 9.21107, 45.48083 9.21089, ..."
1,"POLYGON ((45.47476 9.20507, 45.47474 9.20498, ..."
2,"POLYGON ((45.46766 9.20207, 45.46765 9.20207, ..."
3,"POLYGON ((45.46189 9.19707, 45.46185 9.19687, ..."
4,"POLYGON ((45.45568 9.19307, 45.45567 9.19305, ..."
5,"POLYGON ((45.45247 9.19407, 45.45229 9.19370, ..."
6,"POLYGON ((45.44929 9.19407, 45.44907 9.19365, ..."
7,"POLYGON ((45.44507 9.19507, 45.44527 9.19469, ..."
8,"POLYGON ((45.53649 9.19264, 45.53648 9.19252, ..."
9,"POLYGON ((45.52919 9.19264, 45.52903 9.19230, ..."


In [237]:
# add a column to the `geopandas` data-frame, `newdata`, to identify polygons of the same distance (isochrone)
newdata['new_column'] = [x for x in range(int(newdata.shape[0]/2))]*2
newdata

,geometry,new_column
0,"POLYGON ((45.48083 9.21107, 45.48083 9.21089, ...",0
1,"POLYGON ((45.47476 9.20507, 45.47474 9.20498, ...",1
2,"POLYGON ((45.46766 9.20207, 45.46765 9.20207, ...",2
3,"POLYGON ((45.46189 9.19707, 45.46185 9.19687, ...",3
4,"POLYGON ((45.45568 9.19307, 45.45567 9.19305, ...",4
5,"POLYGON ((45.45247 9.19407, 45.45229 9.19370, ...",5
6,"POLYGON ((45.44929 9.19407, 45.44907 9.19365, ...",6
7,"POLYGON ((45.44507 9.19507, 45.44527 9.19469, ...",7
8,"POLYGON ((45.53649 9.19264, 45.53648 9.19252, ...",0
9,"POLYGON ((45.52919 9.19264, 45.52903 9.19230, ...",1


In [238]:
# `geopandas` data-frame's method `.dissolve(by=<grouping variable>)` merges pairs of polygons together as specified by the values in `new_col` 
newdata = newdata.dissolve(by='new_column')
newdata

,geometry
new_column,
0,"POLYGON ((45.53648 9.19252, 45.53637 9.19227, ..."
1,"POLYGON ((45.52903 9.19230, 45.52891 9.19209, ..."
2,"POLYGON ((45.52157 9.19243, 45.52143 9.19170, ..."
3,"MULTIPOLYGON (((45.51441 9.17459, 45.51449 9.1..."
4,"MULTIPOLYGON (((45.50753 9.17847, 45.50737 9.1..."
5,"MULTIPOLYGON (((45.50366 9.18434, 45.50360 9.1..."
6,"MULTIPOLYGON (((45.50046 9.18455, 45.50058 9.1..."
7,"MULTIPOLYGON (((45.49711 9.18438, 45.49719 9.1..."


In [239]:
# add more columns to describe the table of polygons 
newdata["key"] = mins
newdata["time"] = mins
newdata.head()

,geometry,key,time
new_column,,,
0,"POLYGON ((45.53648 9.19252, 45.53637 9.19227, ...",60,60
1,"POLYGON ((45.52903 9.19230, 45.52891 9.19209, ...",50,50
2,"POLYGON ((45.52157 9.19243, 45.52143 9.19170, ...",40,40
3,"MULTIPOLYGON (((45.51441 9.17459, 45.51449 9.1...",30,30
4,"MULTIPOLYGON (((45.50753 9.17847, 45.50737 9.1...",20,20


In [240]:
# Visualise the merged polygons

# create map
mp = folium.Map([lat,lon],zoom_start=12)

# for each polygon
for i in range(len(cols)):

    polygon = newdata.loc[i, 'geometry']
    
    # check if multipolygon
    try:
        n = len(polygon.geoms)

    except:
        n = 1
        
    # multi-polygons
    if n > 1:
        for j in range(n):
            # polygon
            polygon_j = polygon.geoms[j]
            polygon_j = polygon_j.exterior.coords

            # add isochrone polygon
            folium.Polygon(np.array(list(polygon_j)),
                           tooltip=mins[i],
                           label = mins[i],
                           fill_opacity=0.3,
                           fill_color=cols[i],
                           stroke=.1,
                           color="white").add_to(mp)
    
    # single polygons
    else:
        # add isochrone polygon
        folium.Polygon(np.array(list(polygon.exterior.coords)),
                       tooltip=mins[i],
                       label = mins[i],
                       fill_opacity=0.3,
                       fill_color=cols[i],
                       stroke=.1,
                       color="white").add_to(mp)


# add location marker
folium.Marker([lat,lon],tooltip=address).add_to(mp)

# show map
mp